# GenAI for Humanists
*Capstone project - 2024W 136031-1 GenAI for Humanists*

***

## Generative AI for Automated Text Assessment and Tailored Practice in Spanish Language Learning - *Project Proposal, Felix Aufreiter*

The primary goal of this project is to create a generative AI-driven pipeline that supports Spanish language learners in improving their writing skills. This system will automatically analyse students' written texts, identifying linguistic mistakes and assessing their overall proficiency according to the *Common European Framework of Reference for Languages (CEFRL)* and the available grading schemes of the *BMBWF*.
 
Based on this analysis, the system will generate personalised practice exercises that focus on the specific areas where each student struggles, ensuring targeted and effective learning. These new exercises will be accompanied by detailed solutions. Students will be provided with the possibility to use these newly created tasks to improve their grammar skills. 

Either OpenAI API or a suitable open-source LLM will be selected as the platform for developing this pipeline with detailed prompt engineering. This project should combine generative AI and humanistic education. The aim is to use generative AI to improve the personalised support of students and help educators/teachers to create personalised exercises.

### A short guide to the planned development process.
####  Data Collection
* Collect a dataset of Spanish language learner texts.
* As I work in adult and secondary education myself, I have access to authentic texts from students who currently have a Spanish level of around A1-A2. These texts could be used anonymously for the test of the prompts developed. 
#### Pipeline Development
* **Step 1:** Preprocess student submissions, if necessary with OCR provided by OpenAI API or an open-source model.
* **Step 2:** Detect specific errors (e.g., verb conjugations, word order, prepositions).
* **Step 3:** Use the CEFRL rating scheme to classify texts into proficiency levels. The available rating schemes of the Bundesministerium für Bildung, Wissenschaft und Forschung will be used to grade the texts in accordance with the official guidelines for austrian teachers (Bundesministerium für Bildung, Wissenschaft und Forschung, n.d.).
* **Step 4:** Provide an improved/corrected version of the text . 
* **Step 5:** Use generative AI to design practice materials targeting the identified errors. 
* **Step 6:** Convert the generated exercises to a pdf.
* **Step 7:** Saving the personalised files to a cloud folder with the option to share the newly created files. (This step will only be done if a real-world test is possible.)
#### Expected Outcomes
* Corrected texts with a CEFRL rating according to the learners’ level, corrected texts according to official grading schemes of the Austrian educational system.
* Exercises for the students (pdf with solutions for self-study).
#### Future Use
* The plan would be to use this project in a real-world scenario, because I work in education.
* Furthermore, I am currently developing a relational database which consists of already used tasks for official exams ("Aufgabenpool" - https://aufgabenpool.at/srdp_lfs/index.php?id=sp). These tasks will be adapted to specific needs of the students for further use. Combining the projects of the course "GenAI for Humanists" and the relational database of the "Research Seminar" which also uses an LLM to create new tasks.
*  A great idea would be to connect an LLM to the database to make the search process more userfriendly. 

## Inicial setup
* Step 1: importing OpenAI API-Key
* Step 2: importing packages
* Step 3: creating a function and test prompt to use **OpenAI API**

In [21]:
!pip install openai python-dotenv
!pip install python-docx
!pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [17]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv())

# Access your API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
#Checking directory for '.env' + '.gitignore'
print("Current working directory:", os.getcwd())
print("Contents:", os.listdir(os.getcwd()))

Current working directory: C:\Users\faufr\Documents\GitHub\GenAI_project
Contents: ['.env', '.git', '.gitignore', '.ipynb_checkpoints', 'capstone_project.ipynb', 'README.md']


In [ ]:
# CHECK your API key
print(os.environ.get("OPENAI_API_KEY"))

## Creating test-prompt

In [9]:

from openai import OpenAI
# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [
        {
            "role": "system",
            "content": "You are a very skilled language teacher. Your answers are precise and you use easy language."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


In [11]:
prompt = "Who is 'The Undertaker' and what are his favorite wrestling moves?"
response = get_completion(prompt)
print(response)

The Undertaker is a famous professional wrestler in WWE (World Wrestling Entertainment). His real name is Mark Calaway, and he is known for his dark, mysterious character and impressive performances in the ring. He has been a part of wrestling for over 30 years and is considered one of the greatest wrestlers of all time.

Some of his favorite wrestling moves include:

1. **Tombstone Piledriver**: This is his signature finishing move, where he lifts his opponent upside down and then drops them on their head.

2. **Chokeslam**: He grabs his opponent by the throat and lifts them high before slamming them down to the mat.

3. **Last Ride**: This is a powerful move where he lifts his opponent onto his shoulders and then slams them down.

4. **Old School**: He walks along the top rope and then jumps down onto his opponent.

These moves are part of what makes The Undertaker a legendary figure in wrestling!


## Importing texts
* example texts to analyze with the LLM

In [25]:
#IMPORTING example text
from docx import Document

# Path to the Word document
file_path = './data/correo electronico.docx'

# Open and read the document
doc = Document(file_path)
print(doc)

# Combine all paragraphs into one string
document_text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
print(document_text)

Correo electronico 

De: adrian123@gmail.com
Para: padres123@gmail.com
Fecha: 12 de novembre de 2024
Asunto: me ciudad favorita

Queridos padres,

Estoy en mi ciuadad favorita, Barcelona. La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. ! Fue delicioso! Me gusta de la playa y comer. Luego pasee por el parque y pasee por la playa de la noche. Mas tarde me relaje en la playa de la Barcelona.

Manana tengo planes emocionantes. Quiero visitar muchos mercados y restaurantes para probar mas comida tipica. Tambien espero encontar algunos Souvenirs unicos para llevar a casa.

Estoy disfrutando mucho de mi tiempo aqui y no puedo esperar para contarles mas.

Saludos,
Adrian




## Analyzing the data with an LLM

* The prompt was designed using the `CO-STAR framework`. (https://towardsdatascience.com/how-i-won-singapores-gpt-4-prompt-engineering-competition-34c195a93d41)
***
    * Context: Provide background information on the task
    * Objective: Define what the task is that you want the LLM to perform
    * Style: Specify the writing style you want the LLM to use
    * Tone: Set the attitude of the response
    * Audience: Identify who the response is intended for
    * Response: Provide the response format
***
* The aim of this prompt is to provide examples structure for the LLM to correct the provided text sample ("correo_electrónico" --> document_text). The LLM should use the CEFRL (Common Framework of Reference for Language) to set a bar for the expected quality of the written text which was delivered by a student.
* The actual structre of the feedback is derived from the offical Austrian guidelines for grading students' assignements released by the Austrian **Ministry for Education, Science and Research**:
    * **Description A2-level**:
        * Beurteulungsraster A2 (https://www.bmbwf.gv.at/dam/jcr:ec4e3c97-8d45-4c05-a90d-8d9a764eed7e/reifepr_ahs_mslf_bwr.pdf)
        * Kompetenzbeschreibungen für die zweite lebende Fremdsprache: Französisch, Italienisch, Spanisch – A2 (https://www.matura.gv.at/downloads/download/Kompetenzbeschreibungen%20f%C3%BCr%20die%20zweite%20lebende%20Fremdsprache:%20Franz%C3%B6sisch,%20Italienisch,%20Spanisch%20%E2%80%93%20A2)
    * **Description B1-level**:
        * Bewertungsraster B1 und Begleitmaterialien (überarbeitete Versionen 2023, zu verwenden ab Herbsttermin 2024) (https://www.matura.gv.at/downloads/download/Bewertungsraster%20B1%20und%20Begleitmaterialien%20%28%C3%BCberarbeitete%20Versionen%202023,%20zu%20verwenden%20ab%20Herbsttermin%202024%29)
            * Bewertungsraster B1
            * Begleittext B1
            * Kommentierte Schreibperformanz Spanisch B1

In [29]:
prompt = f"""
Correct the mistakes of the provided text. The text was written by a student. The student should have a language level A2-B1 according
to Common European Framework of Reference for Languages: Learning, Teaching, Assessment CEFRL. Correct the mistakes and provide the adequate solutions.
Don't change the original input text. Only use this structure to correct the text:

Example 1: "Yo te llama José."
Corrected example 1: "Yo te llama José. (Yo me llamo José.)"
Example 2: "Las coches es caros."
Corrected example 2: "Las coches es caros. (Las coches son caros.)"
Example 3: "Hola María, yo te llama José."
Corrected example 3: "Hola María, yo te llama José. (Hola María, me llamo José.) Me gusta los caballo. (Me gustan los caballos.)"

Please correct this text and provide one sentence objective and simple with feedback and which grammar topics the student should improve.
The feedback and explanations have to be in german.
Feedback structure:
"Feedback: [insert feedback sentence here]
An diesem Grammatikthema/diesen Grammatikthemen solltest du noch arbeiten: [insert grammar topics here]
Beurteilungsraster A2:
- Erfüllung der Aufgabenstellung (EA): [insert chosen descriptions]
- Aufbau und Layout (AL): [insert chosen descriptions]
- Spektrum Sprachlicher Mittel (SSM): [insert chosen descriptions]
- Sprachrichtigkeit (SR): [insert chosen descriptions]"
```{document_text}```
"""

response = get_completion(prompt)
print(response)

Correo electronico 

De: adrian123@gmail.com  
Para: padres123@gmail.com  
Fecha: 12 de novembre de 2024  
Asunto: me ciudad favorita  

Queridos padres,  

Estoy en mi ciuadad favorita, Barcelona. (Estoy en mi ciudad favorita, Barcelona.) La ciudad es muy bonita y hay mucha gente viviendo aquí. Además, hace mucho calor aquí en Barcelona.  

Hoy he visto la ciudad. Para el almuerzo probe tapas en un pequeno restaurante local. (Para el almuerzo probé tapas en un pequeño restaurante local.) ! Fue delicioso! Me gusta de la playa y comer. (Me gusta la playa y comer.) Luego pasee por el parque y pasee por la playa de la noche. (Luego paseé por el parque y paseé por la playa de la noche.) Mas tarde me relaje en la playa de la Barcelona. (Más tarde me relajé en la playa de Barcelona.)  

Manana tengo planes emocionantes. (Mañana tengo planes emocionantes.) Quiero visitar muchos mercados y restaurantes para probar mas comida tipica. (Quiero visitar muchos mercados y restaurantes para probar má

## Llamaindex: RAG-pipeline
### Ad data to the pipeline
* This data by the **Ministry for Education, Science and Research** was used to create a `VectorStoreIndex`. The data is stored in the folder `./RAG_data`:
    * **Description A2-level**:
        * Beurteilungsraster A2 (https://www.bmbwf.gv.at/dam/jcr:ec4e3c97-8d45-4c05-a90d-8d9a764eed7e/reifepr_ahs_mslf_bwr.pdf)
        * Kompetenzbeschreibungen für die zweite lebende Fremdsprache: Französisch, Italienisch, Spanisch – A2 (https://www.matura.gv.at/downloads/download/Kompetenzbeschreibungen%20f%C3%BCr%20die%20zweite%20lebende%20Fremdsprache:%20Franz%C3%B6sisch,%20Italienisch,%20Spanisch%20%E2%80%93%20A2)
    * **Description B1-level**:
        * Bewertungsraster B1 und Begleitmaterialien (überarbeitete Versionen 2023, zu verwenden ab Herbsttermin 2024) (https://www.matura.gv.at/downloads/download/Bewertungsraster%20B1%20und%20Begleitmaterialien%20%28%C3%BCberarbeitete%20Versionen%202023,%20zu%20verwenden%20ab%20Herbsttermin%202024%29)
            * Bewertungsraster B1
            * Begleittext B1
            * Kommentierte Schreibperformanz Spanisch B1

**Update your packages to use the pipeline!**

In [ ]:
!pip uninstall llama-index

In [2]:
!pip install llama-index-core llama-index-llms-openai llama-index-embeddings-openai llama-index-readers-file

### Creating a VectorStoreIndex

In [8]:
import os
import openai
from pathlib import Path
from typing import Optional

# 1. Load environment variables from .env
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 2. Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# 3. Import LlamaIndex components
from llama_index.core import (
    VectorStoreIndex,
    Settings,
    StorageContext
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import PDFReader
from llama_index.llms.openai import OpenAI

# Configuration parameters
EMBED_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-3.5-turbo"
TEMPERATURE = 0

def build_pdf_index(pdf_folder: str, persist_dir: Optional[str] = None):
    # Initialize PDF reader
    pdf_loader = PDFReader()
    
    try:
        # 1. Gather PDF files
        folder_path = Path(pdf_folder)
        all_documents = []
        for pdf_file in folder_path.glob("*.pdf"):
            try:
                docs = pdf_loader.load_data(file=pdf_file)
                all_documents.extend(docs)
                print(f"Successfully loaded: {pdf_file.name}")
            except Exception as e:
                print(f"Error loading {pdf_file.name}: {str(e)}")
                continue
        
        print(f"Loaded {len(all_documents)} PDF document chunks.")

        if not all_documents:
            raise ValueError("No documents were successfully loaded")

        # 2. Configure global settings
        Settings.llm = OpenAI(
            model=LLM_MODEL,
            temperature=TEMPERATURE,
            api_key=openai.api_key
        )
        Settings.embed_model = OpenAIEmbedding(
            model=EMBED_MODEL,
            api_key=openai.api_key
        )

        # 3. Create the vector store index
        index = VectorStoreIndex.from_documents(documents=all_documents)

        # 4. Persist the index
        if persist_dir:
            index.storage_context.persist(persist_dir=persist_dir)
            print(f"Index persisted to: {persist_dir}")

        return index

    except Exception as e:
        print(f"Error building index: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        persist_path = "./RAG_index"
        pdf_folder = "./RAG_data"
        
        # Build and persist the index
        print(f"Building new index from {pdf_folder}")
        index = build_pdf_index(pdf_folder=pdf_folder, persist_dir=persist_path)

        # Create a query engine and test it
        query_engine = index.as_query_engine(
            response_mode="compact",
            similarity_top_k=3
        )

        # Run a test query
        user_query = "Please summarize all the key points in these PDFs."
        response = query_engine.query(user_query)
        print("\nAnswer:", response)

    except Exception as e:
        print(f"An error occurred in main: {str(e)}")


Building new index from ./RAG_data
Successfully loaded: bist_lfs_kompetenzbeschreibungen_fr-it-sp_A2_2012-09-17.pdf
Successfully loaded: reifepr_ahs_mslf_bwr.pdf
Successfully loaded: srdp_lfs_Bewertungsraster_B1_2023.pdf
Successfully loaded: srdp_lfs_Bewertungsraster_B1_Begleittext_2023.pdf
Successfully loaded: srdp_lfs_Spanisch_B1_kommentierte_Schreibperformanzen_2023.pdf
Loaded 187 PDF document chunks.
Index persisted to: ./RAG_index

Answer: The PDFs contain competency descriptions for French, Italian, and Spanish at level A2.


### Use the chatbot to ask questions about the data

In [10]:
import os
import openai
from pathlib import Path
from typing import Optional

# 1. Load environment variables from .env
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 2. Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# 3. Import LlamaIndex components
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

# Configuration parameters - updated to use GPT-4
EMBED_MODEL = "text-embedding-ada-002"
LLM_MODEL = "gpt-4"  # or "gpt-4-turbo-preview" for the latest version
TEMPERATURE = 0.7    # Increased for more creative responses

def load_and_query_index(persist_dir: str, query: str):
    try:
        # Configure global settings with GPT-4
        Settings.llm = OpenAI(
            model=LLM_MODEL,
            temperature=TEMPERATURE,
            api_key=openai.api_key
        )
        Settings.embed_model = OpenAIEmbedding(
            model=EMBED_MODEL,
            api_key=openai.api_key
        )

        # Load the existing index
        storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
        index = load_index_from_storage(storage_context=storage_context)

        # Create an advanced query engine with additional parameters
        query_engine = index.as_query_engine(
            response_mode="tree_summarize",  # Provides more structured responses
            similarity_top_k=5,              # Increased for broader context
            streaming=True                   # Enable streaming responses
        )

        # Execute the query
        response = query_engine.query(query)
        return response

    except Exception as e:
        print(f"Error querying index: {str(e)}")
        raise

def interactive_query_session():
    persist_path = "./RAG_index"
    
    print("\nWelcome to the Interactive RAG Query System")
    print("Type 'exit' to end the session")
    print("----------------------------------------")

    while True:
        try:
            # Get user input
            user_query = input("\nEnter your question: ").strip()
            
            # Check for exit command
            if user_query.lower() == 'exit':
                print("Ending session. Goodbye!")
                break
            
            # Process query and get response
            if user_query:
                print("\nProcessing your query...")
                response = load_and_query_index(persist_path, user_query)
                print("\nAnswer:", response)
            else:
                print("Please enter a valid query.")

        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please try again with a different query.")

if __name__ == "__main__":
    try:
        interactive_query_session()
    except KeyboardInterrupt:
        print("\nSession terminated by user. Goodbye!")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")



Welcome to the Interactive RAG Query System
Type 'exit' to end the session
----------------------------------------



Enter your question:  About which levels of spanish to do have information?



Processing your query...

Answer: The information available pertains to the A2 level of Spanish.



Enter your question:  Is there also information about B1?



Processing your query...

Answer: Yes, there is information about B1. It outlines skills such as the ability to write simple, coherent texts on familiar topics, interact in writing by conveying simple, immediate information, and express personal importance. At this level, individuals can recount uncomplicated stories or descriptions fluently, and share thoughts on both abstract and concrete topics. They can also write reports and essays on topics of general interest, using simple language to list pros and cons, express and justify their own opinions. For vocabulary, B1 level individuals show a good command of basic vocabulary but may still make elementary mistakes when expressing more complex matters or unfamiliar topics. They can use a wide range of simple words appropriately when discussing familiar topics. As for grammatical correctness, they can communicate adequately in familiar situations and use a repertoire of frequently used phrases and expressions correctly. They can also gi


Enter your question:  Exit


Ending session. Goodbye!


In [12]:
# You can also use it programmatically:
persist_path = "./RAG_index"
query = "What are the main themes discussed in these documents?"
response = load_and_query_index(persist_path, query)
print(response)

The main themes discussed in these documents are the assessment and grading of written performances in Spanish. The documents detail the evaluation of tasks that involve writing informative articles for a young audience interested in intercultural exchange. The evaluation criteria include the fulfillment of the task, coherence and cohesion of the text, the use of appropriate language and style, and the treatment of specific content points. The documents also address issues such as the appropriateness of the title and introduction, the balance of content points, and the clarity of the writer's role.


## Now have a look at the finished version stored in the Jupyter-Notebook `FAST_VERSION.ipynb`!

### 2nd RAG-pipeline for task-creation
* The collection of tasks was privatly assembled and includes grammar exercises. These exercises were used to create a `VectorStoreIndex`. The data is stored in the folder `./TASK_RAG_data`:

In [ ]:
#ADD second vectorestoreindex

In [ ]:
#ADD task creation prompts